# Limpieza de Hulu

## Pedir datos a la API

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq

import sys, os
sys.path.append(os.path.abspath(".."))

from src.buscar_peliculas import buscar_peliculas

Pedimos los datos:

In [3]:
dfhulu_raw = buscar_peliculas('Hulu')

<Response [200]> 5 1077


Fecha de consulta en la API: 18-10-2025 23:39

Verificamos que no haya perdida de datos:

In [4]:
largo_dataframe = len(dfhulu_raw[0])
largo_segun_API = dfhulu_raw[1]
print([largo_dataframe, largo_segun_API])

[1077, 1077]


Los largos coinciden

In [5]:
dfhulu1 = dfhulu_raw[0]

In [6]:
dfhulu1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,1874486,Predator: Killer of Killers,2025,tt36463894,1376434,movie,movie
1,1615420,Bullet Train,2022,tt12593682,718930,movie,movie
2,1583908,The Menu,2022,tt9764362,593643,movie,movie
3,1515072,Uncharted,2022,tt1464335,335787,movie,movie
4,1650358,Barbarian,2022,tt15791034,913290,movie,movie
...,...,...,...,...,...,...,...
1072,1589801,Adrift,2018,tt3454104,635185,movie,movie
1073,1630649,The Making and Meaning of 'We Are Family',2002,tt0306219,806685,movie,movie
1074,1606945,Macross FB7: Listen to My Song!,2012,tt3709868,258156,movie,movie
1075,538039,Robert Durst: An ID Murder Mystery,2019,tt9726390,593962,movie,movie


Guardamos estos datos crudos en un csv

In [7]:
dfhulu1.to_csv('../data/raw_api_hulu.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `type` y `tmdb_type` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [8]:
dfhulu1['type'].unique()

array(['movie'], dtype=object)

In [9]:
dfhulu1['tmdb_type'].unique()

array(['movie'], dtype=object)

Al ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [10]:
dfhulu2 = dfhulu1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores Na

In [11]:
resultados = {}
for i in dfhulu2.columns:
    resultados[i] = dfhulu2[i].isna().sum()
resultados

{'id': np.int64(0),
 'title': np.int64(0),
 'year': np.int64(0),
 'imdb_id': np.int64(0),
 'tmdb_id': np.int64(0)}

### Duplicados

In [12]:
dfhulu2.duplicated().sum()

np.int64(0)

#### imdb_id

In [14]:
dfhulu2.duplicated('imdb_id').sum()

np.int64(0)

#### id
Deberia ser unico para cada pelicula

In [15]:
dfhulu2.duplicated('id').sum()

np.int64(0)

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [16]:
dfhulu2.duplicated('title').sum()

np.int64(12)

revisemos porque se repiten los titulos de las peliculas

In [17]:
dfhulu2[dfhulu2.duplicated('title')].head()

,id,title,year,imdb_id,tmdb_id
194,1397596,The Host,2006,tt0468492,1255
209,1336403,Scream,1996,tt0117571,4232
469,11040148,Stay,2025,tt38039379,1547527
652,1615592,Robots,2023,tt12579470,723347
662,1596325,Alone,2020,tt10192566,661950


In [18]:
dfhulu2[dfhulu2['title'] == 'Scream']

,id,title,year,imdb_id,tmdb_id
11,1607086,Scream,2022,tt11245972,646385
209,1336403,Scream,1996,tt0117571,4232


Son distintas peliculas con mismo nombre

### Tipos de columnas

In [19]:
dfhulu2.dtypes

id          int64
title      object
year        int64
imdb_id    object
tmdb_id     int64
dtype: object

Los tipos de las columnas estan bien

### Analisis de columna `year`

In [20]:
dfhulu2['year'].describe()

count    1077.000000
mean     2016.824513
std         9.174974
min      1964.000000
25%      2012.000000
50%      2021.000000
75%      2023.000000
max      2025.000000
Name: year, dtype: float64

todos estan dentro de un rango logico

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [22]:
dfhulu2.to_csv('../data/api_hulu.csv', index=False)